In [ ]:
day_obs_min = "Yesterday"
#day_obs_min = "2024-12-05"
day_obs_max = "Yesterday"
#day_obs_max = "2024-12-05"
time_order = 'newest first'
show_salIndex = 'all'
show_table = True
show_timeline = False

tokenfile = "/Users/lynnej/.lsst/rsp_prod"
site = "usdf"

In [ ]:
import base64
import warnings
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

from astropy.time import Time, TimeDelta
import yaml
import html

from rubin_nights.scriptqueue import get_consolidated_messages

In [ ]:
## This cell is all about formatting and presentation of completed scriptqueue

# Define name and colours from salIndex    
def get_name_and_color_from_salindex(sal_index, unknown_color='#f9f9f9'):
    # Colors from https://medialab.github.io/iwanthue/
    return {
        5: ('Simonyi Exposure', '#b6ecf5'),
        6: ('Auxtel Exposure', '#d8f1f5'),
        0: ('Narrative log', '#cf7ddc'),
        1: ('MTQueue', '#b4c546'),
        2: ('ATQueue', '#bab980'),
        3: ('OCSQueue', '#b2baad'),
        4: ('EFD error', '#9cb5d5'),
    }.get(sal_index, ("??", unknown_color))
    
# Add a custom formatter to handle YAML-like strings with dynamic background colors
def format_config_as_yaml_with_colors(row):
    config_value = row['config']
    sal_index = row['salIndex']
    script_salindex = row['script_salIndex']
    
    # Define background colors based on salIndex
    background_color = get_name_and_color_from_salindex(sal_index)[1]

    might_be_yaml = (script_salindex > 0) and (sal_index in [1, 2, 3])
    might_be_yaml = might_be_yaml and isinstance(config_value, str) and len(config_value) > 0
    might_be_yaml = might_be_yaml and not config_value.startswith('Traceback')

    #if script_salindex > 0 and sal_index in [1,2,3] and isinstance(config_value, str) and len(config_value) > 0:
    if might_be_yaml:
        try:
            # Parse the YAML-like string
            parsed_yaml = yaml.safe_load(config_value)
            # Format back to YAML with proper indentation
            formatted_yaml = yaml.dump(parsed_yaml, default_flow_style=False)
            return (
                f"<pre style='background: {background_color}; padding: 10px; border: 1px solid #ddd; margin: 0;'>"
                f"{formatted_yaml}</pre>"
            )
        except yaml.YAMLError:
            # If parsing fails, return as plain text in a styled <pre> block
            return (
                f"<pre style='background: {background_color}; padding: 10px; border: 1px solid #ddd; margin: 0;'>"
                f"{config_value}</pre>"
            )
    elif config_value.startswith('Traceback'):
        return f"<pre style='background: {background_color}'>{config_value}</pre>"
    else:
        return config_value  # Return as-is if salIndex is 0 or invalid type

def format_description(row):
    if row.description.startswith("<a href"):
        return row.description
    else:
        # Escape these in particular because of tracebacks
        return html.escape(row.description)

    
def pretty_print_messages(efd_and_messages: pd.DataFrame, cols: list, time_order: str,
                         show_salIndex: list[int] = [0, 1, 2, 3, 4, 5]) -> None:

    keep = np.zeros(len(efd_and_messages), dtype=bool)
    for si in show_salIndex:
        keep |= efd_and_messages.salIndex == si
    efd_and_messages = efd_and_messages[keep]
    
    def highlight_salindex(s):
        return [f'background-color: {get_name_and_color_from_salindex(s.salIndex)[1]}'] * len(s)
    msg = ["Color coding by "]
    for i in np.sort(efd_and_messages.salIndex.unique()):
        what, color = get_name_and_color_from_salindex(i)
        msg.append(f" <font style='background-color: {color[0:]};'>{what}</font> ")
    display(HTML(" ".join(msg)))
    
    if time_order == "newest first": 
        efd_and_messages = efd_and_messages[::-1]
    # Apply yaml-like formatting conditionally
    efd_and_messages['config'] = efd_and_messages.apply(format_config_as_yaml_with_colors, axis=1)
    efd_and_messages['description'] = efd_and_messages.apply(format_description, axis=1)
    # Adjust the display call to include the formatted column
    styled_table = (
        efd_and_messages[cols]
        .style.apply(highlight_salindex, axis=1)  # Preserve color formatting for other columns
        .set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
        .set_properties(**{'text-align': 'left'})
    )
    
    # Render with HTML
    display(HTML(styled_table.format().to_html(render_links=True)))
    return 

In [ ]:
# Set a range of times to search, based on dayobs
if day_obs_min.lower() == "today":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/    
    # Drop the hours, minutes, seconds to get the ISO formatted day_obs
    day_obs_min = Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc').iso[0:10]

if day_obs_min.lower() == "yesterday":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/
    # Drop the hours, minutes, seconds to get the ISO fromatted day_obs
    day_obs_min = (Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc') - TimeDelta(1, format='jd')).iso[0:10]

# Set a range of times to search, based on dayobs
if day_obs_max.lower() == "today":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/    
    # Drop the hours, minutes, seconds to get the ISO formatted day_obs
    day_obs_max = Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc').iso[0:10]

if day_obs_max.lower() == "yesterday":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/
    # Drop the hours, minutes, seconds to get the ISO fromatted day_obs
    day_obs_max = (Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc') - TimeDelta(1, format='jd')).iso[0:10]

try:
    t_start = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
except ValueError:
    print(f"Is day_obs_min the right format? {day_obs_min} should be YYYY-MM-DD")
    t_start = None
try:
    t_end = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc') + TimeDelta(1, format='jd')
except ValueError:
    print(f"Is day_obs_max the right format? {day_obs_max} should be YYYY-MM-DD")
    t_start = None

if t_start is None or t_end is None:
    print("Did not get valid inputs for time period.")


# t_start = Time("2025-03-03 20:50:28.663076", format='iso', scale='utc')
# t_end = Time('2025-03-03 21:06:28.663076', format='iso', scale='utc') + TimeDelta(1, format='sec')

print(f"Querying for messages from {t_start.iso} to {t_end.iso}")
print(f"Notebook executed at {Time.now().utc.iso}")
efd_and_messages, cols = get_consolidated_messages(t_start, t_end, tokenfile=tokenfile, site=site)

# Could add these to parameters
save_log = False
make_link = False

if save_log:
    log_filename = f"log_{day_obs_min}_{day_obs_max}.h5"
    # We will always get a performance warning here, because the dataframe includes string objects
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        efd_and_messages[cols].to_hdf(log_filename, key='messages')
        print(f"Wrote to {log_filename}")
if make_link:
    import base64
    html_table = efd_and_messages[cols].to_xml(index=False)
    b64 = base64.b64encode(html_table.encode())
    payload = b64.decode()
    log_xml =  f"log_{day_obs_min}_{day_obs_max}.xml"
    html_link = f'<a download="{log_xml}" href="data:text/csv;base64,{payload}" target="_blank">Download XML table of log messages</a>'
    display(HTML(html_link))
    print(" read download with pandas.read_xml, convert times using .astype('datetime64[ns]')")

In [ ]:
if isinstance(show_salIndex, str) and show_salIndex.lower() == 'all':
    show_salIndex = efd_and_messages.salIndex.unique()
# Ok, otherwise we have to do some parsing .. we get a string but need list of ints.
showsal = []
for i in show_salIndex:
    try:
        showsal.append(int(i))
    except ValueError:
        # Wasn't an integer, pass
        # easier when no negative int salIndexes.
        pass
show_salIndex = showsal

if show_table:
    pretty_print_messages(efd_and_messages, cols, time_order, show_salIndex=show_salIndex)